In [1]:
from pyiron_contrib.workflow.function import function_node
from pyiron_contrib.workflow.workflow import Workflow
from pyiron_contrib.workflow.function import Function
import matplotlib.pyplot as plt
import numpy as np
import random
from pyiron import Project

- Add a metadata file for each node step; including all possible info with DOIs
- Workflow top-level metadata - the matwerk yaml file can be used
- Execution metadata
- What would go into the method section?

In [2]:
def create_structure(element=None):
    """
    Create an alloy structure
    """
    pr = Project("elastic_constant")
    structure = pr.create.structure.bulk(element, 
                                         cubic=True)
    return structure

In [3]:
create_structure_node = Function(create_structure, "structure", 
                                 update_on_instantiation=False)

In [4]:
def calculate_elastic_matrix(structure=None,
                            potential='2009--Mendelev-M-I--Al-Mg--LAMMPS--ipr1',
                            eps_range=0.001):
    pr = Project('elastic_constant')
    job = pr.create.job.Lammps(job_name="lammps_job")
    job.structure = structure
    job.potential = potential
    job.calc_minimize()
    elastic_job = job.create_job(pr.job_type.ElasticMatrixJob, "elastic_job")
    elastic_job.input["eps_range"] = eps_range
    elastic_job.run()
    return elastic_job["output/elasticmatrix"]["C"]

In [5]:
elastic_matrix_node = Function(calculate_elastic_matrix, "elastic_matrix", 
                                 update_on_instantiation=False)

In [22]:
create_structure_node.inputs.element = 'Al'
elastic_matrix_node.inputs.structure = create_structure_node.outputs.structure

The job elastic_job was saved and received the ID: 740
The job s_e_0 was saved and received the ID: 741
The job s_01_e_m0_00100 was saved and received the ID: 742
The job s_01_e_m0_00050 was saved and received the ID: 743
The job s_01_e_0_00050 was saved and received the ID: 744
The job s_01_e_0_00100 was saved and received the ID: 745
The job s_08_e_m0_00100 was saved and received the ID: 746
The job s_08_e_m0_00050 was saved and received the ID: 747
The job s_08_e_0_00050 was saved and received the ID: 748
The job s_08_e_0_00100 was saved and received the ID: 749
The job s_23_e_m0_00100 was saved and received the ID: 750
The job s_23_e_m0_00050 was saved and received the ID: 751
The job s_23_e_0_00050 was saved and received the ID: 752
The job s_23_e_0_00100 was saved and received the ID: 753


/home/menon/miniconda3/envs/workflow-desc/lib/python3.10/site-packages/pyiron_contrib/workflow/channels.py:272: UserWarning: structure (InputData) and structure (OutputData) were not a valid connection
  warn(


In [25]:
elastic_matrix_node.outputs.elastic_matrix.value

array([[105.90187899,  60.85060019,  60.85060019,   0.        ,
          0.        ,   0.        ],
       [ 60.85060019, 105.90187899,  60.85060019,   0.        ,
          0.        ,   0.        ],
       [ 60.85060019,  60.85060019, 105.90187899,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,  31.46945444,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
         31.46945444,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,  31.46945444]])